In [50]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt

In [51]:
# !pip install --upgrade pythainlp
# !pip install pyLDAvis

In [52]:
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings


In [53]:
df = pd.read_csv("sentimentMacbookAirM1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  81 non-null     object
dtypes: object(1)
memory usage: 776.0+ bytes


In [54]:
df = df.astype(str)
df

,message
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก
...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...


# Tokenize word

In [55]:
stop_word = list(pythainlp.corpus.thai_stopwords())
rm_word = [ ' ','    ','\n', '(', ')', '?', '+', 'ผม', ':', 'ตัว', 'เรื่อง', 'ส่วนตัว', 'คับ', '--', '//', 'แน่นอน', 'สอย', 'ซื้อ', 'เวลา', '' ]
screenword = stop_word + rm_word

def tokenize_n_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize( str(sentence), engine = 'newmm' )
    for i in words :
        if i not in screenword :
            merged = merged + ',' + i
    return merged[1:]

In [56]:
df['tokenized'] = df['message'].apply(lambda x: tokenize_n_space(x))
df

,message,tokenized
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ..."
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot..."
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม"
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส..."
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก"
...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,..."
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว..."
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน..."
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,..."


# Creating Dictionary

In [57]:
doc = df['tokenized'].to_list()
texts = [ [i for i in dc.split(',')] for dc in doc ]

dictionary = gensim.corpora.Dictionary(texts)

print(dictionary.token2id.keys())

dict_keys(['1', 'Macbook', 'Performance', 'Windows', 'air', 'drop', 'm', 'คอนเฟิร์ม', 'ลืม', 'ลื่นไหล', 'สักพัก', 'อึด', 'เสียง', 'แบต', 'illustrator', 'mac', 'photoshop', 'กราฟิก', 'กะ', 'ตัดต่อ', 'ทำงาน', 'น้อง', 'สอบถาม', 'ไหว', 'M', 'พัดลม', 'ระดับ', 'เทพ', 'โปรแกรม', 'ๆๆๆ', '28000', 'การใช้งาน', 'ดี', 'ต่อให้', 'ทำ', 'ทำได้', 'ประสบการณ์', 'ราคา', 'ห่า', 'เกม', 'เล่น', 'โน๊ตบุ๊ค', 'โลก', 'จุดเด่น', 'ดีมาก', 'ลำโพง', '7', 'CTW', 'geniusbar', 'studio', 'ขาย', 'คนอื่น', 'คอม', 'จี', 'ดีกว่า', 'ดู', 'ที้ง', 'นิดนึง', 'บริการ', 'พนักงาน', 'ระบาย', 'รุ', 'ร้าน', 'สอง', 'สัก', 'สาขา', 'ห่วย', 'เข้าไป', 'เจอ', 'เนาะ', 'เนียส', 'แย่', 'แอปเปิ้ล', 'ใช้งาน', '2013', '2020', '2023', '34', '900', 'Air', 'คน', 'ค่ำ', 'งานเอกสาร', 'ตอน', 'ต้นปี', 'ประชุม', 'ประทับใจ', 'ปี', 'มีปัญหา', 'ยัน', 'รุ่น', 'หาย', 'เก่า', 'เครื่อง', 'เดิม', 'เปิดตัว', 'เลือก', 'แชร์', 'แต่เช้า', 'ไว', '/', '4', 'adobe', 'apple', 'exel', 'premia', 'word', 'การตัดต่อ', 'ขั้นพื้นฐาน', 'ความแตกต่าง', 'ค่', 'ง', 'จ่าย', 'ดร'

In [58]:
gensim_corpus = [ dictionary.doc2bow(tx, allow_update = True) for tx in texts ]
word_frq = [ [ (dictionary[ides], frq) for ides, frq in coup ] for coup in gensim_corpus ]

# Topic Modeling

In [59]:
num_topic = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_number = 1 # more number, more compute time spending

# Make index to word dictionary
temp = dictionary[0]
id2word = dictionary.id2token

%time
model = gensim.models.LdaModel( corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto'
                               , iterations=iterations, num_topics=num_topic, passes=passes, eval_every=eval_number )

Wall time: 0 ns


In [60]:
pyLDAvis.gensim.prepare( model, gensim_corpus, dictionary )

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.062582 -0.059201       1        1  10.423606
10    -0.162539 -0.154053       2        1   7.874873
3      0.015149 -0.152504       3        1   7.638766
20     0.207807  0.038180       4        1   5.971836
29     0.018933  0.018849       5        1   5.841249
26     0.076564 -0.090533       6        1   5.608388
16     0.001265 -0.060338       7        1   5.469022
8      0.017926 -0.006841       8        1   5.105824
9      0.082795  0.073960       9        1   4.890133
4     -0.058859  0.150645      10        1   4.601461
15    -0.094983  0.120448      11        1   4.388436
24     0.054116 -0.023834      12        1   4.104762
17     0.002625  0.054975      13        1   3.205667
14    -0.063918  0.026746      14        1   2.913126
19    -0.121170  0.006748      15        1   2.870188
13    -0.002086 -0.055494      16        1   2.703497
0     -0.036223  0.091549      17        1   2.623942
11     0.067149 -0.029590      18        1   2.445868
1      0.018733  0.029621      19        1   2.412650
25     0.055278 -0.006866      20        1   2.283734
23     0.023896 -0.021607      21        1   1.678725
28     0.030648 -0.058010      22        1   1.018827
7     -0.014950 -0.042925      23        1   0.884779
18    -0.043898  0.040149      24        1   0.675583
22     0.018284 -0.003892      25        1   0.674913
2     -0.028962  0.021278      26        1   0.569880
21     0.036018  0.014580      27        1   0.517372
27    -0.003171  0.012385      28        1   0.466255
6     -0.016922  0.032788      29        1   0.068564
12    -0.016922  0.032788      30        1   0.068072, topic_info=       Term       Freq      Total Category  logprob  loglift
4       air  19.000000  19.000000  Default  30.0000  30.0000
0         1  33.000000  33.000000  Default  29.0000  29.0000
24        M  18.000000  18.000000  Default  28.0000  28.0000
6         m  20.000000  20.000000  Default  27.0000  27.0000
28  โปรแกรม  10.000000  10.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
23      ไหว   0.001478   2.994749  Topic30  -6.7731  -0.3214
24        M   0.001478  18.145285  Topic30  -6.7731  -2.1229
25    พัดลม   0.001478   7.640106  Topic30  -6.7731  -1.2579
26    ระดับ   0.001478   3.167767  Topic30  -6.7731  -0.3776
28  โปรแกรม   0.001478  10.736162  Topic30  -6.7731  -1.5981

[1448 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
169       4  0.208346       
169       6  0.208346       
169       8  0.208346       
169       9  0.138897       
169      10  0.208346       
...     ...       ...    ...
306       9  0.273693  ไอแพด
266      13  0.799733  ไอโฟน
825       1  0.865840     ๆ?
833      16  0.822560      ”
844       6  0.739866    🥹😭 

[1279 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 11, 4, 21, 30, 27, 17, 9, 10, 5, 16, 25, 18, 15, 20, 14, 1, 12, 2, 26, 24, 29, 8, 19, 23, 3, 22, 28, 7, 13])

In [61]:
model.show_topic(1)

[('นิ', 0.054400925),
 ('น', 0.05440077),
 ('เล่น', 0.040913258),
 ('คุ้ม', 0.04091315),
 ('เกม', 0.04091234),
 ('ดีกว่า', 0.027425494),
 ('ประสบการณ์', 0.027425447),
 ('เท', 0.027423693),
 ('ทำงาน', 0.01394296),
 ('เพลง', 0.013940051)]

In [62]:
df['topics'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [63]:
df

,message,tokenized,topics,score
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ...",23,0.958951
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot...",10,0.956374
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม",27,0.930099
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส...",8,0.967957
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก",24,0.870768
...,...,...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,...",13,0.965871
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว...",26,0.980566
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน...",9,0.986846
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,...",29,0.981122
